# DATA SETS

* FIFA 21: Archivo *=>* [ FIFA-21 Complete.csv ](https://www.kaggle.com/datasets/aayushmishra1512/fifa-2021-complete-player-data?resource=download)

El data set original cuenta con 9 columnas: "player_id", "name", "nationality", "position", "overall", "age", "hits", "potential" y "team". En ese orden.

El objetivo es obtener un archivo csv, con las columnas: "Team", "Nationality", "Position", "Age", "Potential" y "Name". En ese orde. Sin embargo, antes, se debe hacer algunas conversiones de datos para poder ser utilizados.

* Traducción de las posiciones de los jugadores, del ingles al español. "DK" debe pasar a ser "Portero", por ejemplo.

Para ello, se utiliza la siguiente función: 

In [1]:
#imports necesarios
import csv
import json
from os import path #utilizado para obtener la rutas de archivos

In [2]:
def traducir_pos(pos):
    """
    :param pos: la posicion en ingles de un jugador
    :return: la traduccion de la posicion
    """
    # Se almacena, en un diccionario, la traduccion (valor) 
    # de la posicion (clave)
    try:
        with open('fifa_pos_ENG_ESP.json', 'r', encoding="utf-8") as traduccion:
            trad = json.load(traduccion)
            return trad[pos]
    except FileNotFoundError:
        return ("No encontrado")

* Reemplazar el valor de 'potential', que contiene un valor numerico fijo, a su equivalente en palabras.

Para ello, se utiliza la siguiente función: 

In [3]:
def conversion_del_potencial(potencial):
    """
    :param potencial: el valor de la columna 'Potential'
    :return: el equivalente a potencial en string
    """
    if(potencial < "60"):
        potencial = "Regular"
    elif(potencial > "60" and potencial <= "79"):
        potencial = "Bueno"
    elif(potencial > "79" and potencial <= "89"):
        potencial = "Muy bueno"
    else:
        potencial = "Sobresaliente"
    return potencial

* Las funciones anteriores, son implementadas en otra función que recibirá el iterador del csv y aplicará, a cada fila, las anteriores funciones. Retornando el resultado en una lista.

In [4]:
def filtro_fifa(csvreader):
    """
    :param csvreader: el contenido del archivo en una lista
    :return: una lista filtrada segun los criterios adoptados
    """
    lista_filtrada = []
    for ln in csvreader:
        # conversion de numero a strig segun el potencial
        ln[7] = conversion_del_potencial(ln[7])
        # traduccion de las posiciones
        # se aplica un split ya que, un jugador, puede tener varias posiciones 
        ln[3] = '|'.join(
            list(map(lambda pos: traducir_pos(pos), ln[3].split('|'))))
        lista_filtrada.append(ln)

    return lista_filtrada

* Teniendo lo anterior en cuenta, solo queda reacomodar las columnas y descartar aquellas que no serán utilizadas.

Para ello se utiliza la siguiente funcion:

In [5]:
def reacomodar_columnas(lista_filtrada):
    """
    :param lista_filtrada: lista retornada por la funcion [filto_fifa(csvreader)]
    :return: la misma lista pero con las columnas intercambiadas y algunas eliminadas
    """
    # “Team”, “Nationality”, “Position”, “Age”, “Potential” y “Name”
    for ln in lista_filtrada:
        # a, b = b, a # Swapping
        ln[8], ln[0] = ln[0], ln[8]
        ln[2], ln[1] = ln[1], ln[2]
        ln[3], ln[2] = ln[2], ln[3]
        ln[5], ln[3] = ln[3], ln[5]
        ln[7], ln[4] = ln[4], ln[7]
        # name ya quedo en la posicion que le corresponde
    lista_filtrada = list(map(lambda ln: ln[0:6], lista_filtrada))
    return lista_filtrada

* Invocación de todo lo anterior, para luego generar un nuevo archivo csv que cumpla con las condiciones planteadas para ser utilizado.

In [6]:
try:
    with open("FIFA-21 Complete.csv", 'r', encoding="utf-8") as fifa:
        csvreader = csv.reader(fifa, delimiter=';')
        next(csvreader)  # omito el encabezado
        lista_filtrada = filtro_fifa(csvreader)
        lista_filtrada = reacomodar_columnas(lista_filtrada)

    ruta_completa = path.join(path.realpath('..'), "dataset_fifa21.csv")

    with open(ruta_completa, 'w', encoding="utf-8", newline='') as fifa:
        writer = csv.writer(fifa)
        # Escribo el nuevo encabezado
        writer.writerow(["Team", "Nationality", "Position", "Age", "Potential", "Name"])
        for ln in lista_filtrada:
            writer.writerow(ln)
except FileNotFoundError:
    print("Archivo csv 'FIFA-21 complete.csv' no encontrado")